In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import torch
import torch.nn as nn

from torchinfo import summary
from experiments.cv.models import MLP, LeNet


net = nn.Linear(28 * 28, 10)

In [2]:
params = dict(net.named_parameters())
buffers = dict(net.named_buffers())

In [26]:
import torch
from torch import Tensor
from torch.func import functional_call, jacrev, vmap, grad
from overrides import overrides
from absl import logging
from tqdm.auto import tqdm

def _fnet_single(params, buffers, model, x):
    return functional_call(model, (params, buffers), (x,))

loss_fn = nn.CrossEntropyLoss()

def loss_fn_single(params, buffers, model, x, y):
    return loss_fn(_fnet_single(params, buffers, model, x), y)


def _compute_phi(
        model,
        params,
        buffers,
        data: Tensor,
        batch_size: int = 128,
        dtype: torch.dtype = torch.float32,
        device: str | torch.device = (
                "cuda" if torch.cuda.is_available() else "cpu"),
        *args,
        **kwargs,
):
    logging.info("Computing the kernel matrix, this may take a while...")

    n = data.shape[0]
    d = sum(p.numel() for p in params.values())
    print('d', d)

    PHI = torch.zeros((n, d), dtype=dtype, device=device)
    K = torch.zeros((n, n), dtype=dtype, device=device)

    for i in tqdm(range(0, n, batch_size), leave=False):
        j = min(i + batch_size, n)
        grads = vmap(grad(loss_fn_single), (None,None,None, 0, 0))(params, buffers, model, x[i:j], y[i:j])
        PHI[i:j] = torch.cat(
            [g.reshape(j-i, -1) for g in grads.values()], dim=1
        )  # (batch_size, d)
        # PHI[i:j] = torch.stack()
        # print([j.numel() / 20 for j in jac.values()])
        # print(jac)
        # print(type(jac))
        # PHI[i:j] = vmap(jacrev(_fnet_single), (None,None,None, 0))(params, buffers, model, x[i:j])
        K[i:j, i:j] = PHI[i:j] @ PHI[i:j].T
    return K

x = torch.rand(20, 1, 28* 28)
y = torch.randint(0, 10, (20,1))

K = _compute_phi(net, params, buffers, x)
print(torch.linalg.matrix_rank(K))

d 7850


  0%|          | 0/1 [00:00<?, ?it/s]

tensor(20, device='cuda:0')
